In [1]:
] status

Status `~/.julia/environments/v1.9/Project.toml`
  [336ed68f] CSV v0.10.11
  [159f3aea] Cairo v1.0.5
  [a81c6b42] Compose v0.9.5
  [a93c6f00] DataFrames v1.6.1
⌃ [4b033969] DiscreteValueIteration v0.4.6
  [186bb1d3] Fontconfig v0.4.1
  [38e38edf] GLM v1.9.0
  [a2cc645c] GraphPlot v0.5.2
  [86223c79] Graphs v1.9.0
  [7073ff75] IJulia v1.24.2
⌃ [7588e00f] POMDPTools v0.1.5
⌃ [a93abf59] POMDPs v0.9.5
  [91a5bcdd] Plots v1.39.0
  [8af83fb2] QuickPOMDPs v0.2.14
  [bf21e494] Shuffle v0.1.1
  [276daf66] SpecialFunctions v2.3.1
  [2913bbd2] StatsBase v0.34.2
  [b4f28e30] TikzGraphs v1.4.0
  [37f6aa50] TikzPictures v3.5.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [2]:
using CSV
using DataFrames
using Printf
# basic structure for Markov Decision Problems
mutable struct MDP
    γ::Float64
    S::UInt64
    A::UInt64
    T 
    R # map (state, action) to reward 
    samples # map (state, action) to a Dict mapping each state to its count
end

# loading data
# filename: 
# S: an integer, representing the size of the state space
# A: an integer, representing the size of the action space
# gamma: the discount trate
# guess: a function that completes the T and R matrix based on samples 
function load_from_file(filename, S, A, γ)
    df = CSV.read(filename, DataFrame)
    return load_from_df(df, S, A, γ)
end

function load_from_df(df, S, A, γ)
    # initialize the sample and reward array
    samples = Dict()
    rewards = Dict()
    for s=1:S
        for a=1:A
            samples_s_a = Dict()
            df_s_a = df[(df.s.==s) .& (df.a.==a), :]
            if size(df_s_a, 1) > 0
                rewards[(s, a)] = df_s_a[1, :r]
                df_s_a_count = combine(groupby(df_s_a, :sp), nrow)
                for i=1:size(df_s_a_count,1)
                    samples_s_a[df_s_a_count[i, :sp]] = df_s_a_count[i,:nrow]
                end
                samples[(s, a)] = samples_s_a
            else
                throw(error("missing samples"*string(s)*","*string(a)))
            end
        end
    end
    return MDP(γ, S, A, Dict(), rewards, samples)
end

load_from_df (generic function with 1 method)

In [3]:
# completing the Transition matrix using max likelihood
function complete_T_MLE(P::MDP)
    T = Dict()
    for s=1:P.S
        for a=1:P.A
            T_s_a = Dict()
            samples = P.samples[s, a]
            ttl = sum([cnt for (sp, cnt) in samples])
            for (sp, cnt) in samples
                T_s_a[sp] = cnt / ttl
            end
            T[(s,a)] = T_s_a
        end
    end
    P.T = T
end

# one step lookahead, based on a certain utility function
# given MDP P and utility U, output best policy and utility from state s
function lookahead(P::MDP, U, s)
    Q_s = Dict()
    for a=1:P.A
        Q_s[a] = P.R[(s,a)] + P.γ * sum([prob * U[sp] for (sp, prob) in P.T[(s,a)]])
    end
    # update
    a_s, U_s = -1, -Inf
    for a=1:P.A
        if Q_s[a] > U_s
            a_s = a
            U_s = Q_s[a]
        end
    end
    return a_s, U_s 
end

# solving small via value iteration
function value_iteration(P::MDP, iter::Int64)
    # initialize utility function
    # utility function
    U = Dict()
    for s=1:P.S
        U[s] = 0
    end
    # bellman update
    # looka
    
    for it=1:iter
        for s=1:P.S
            # compute action-value funciton
            a_s, U_s = lookahead(P,U,s)
            U[s] = U_s
        end
    end

    # output policy based on utility
    policy = Dict()
    for s=1:P.S
        a_s, U_s = lookahead(P,U,s)
        policy[s] = a_s
    end

    return U, policy
end


P_small = load_from_file("data/small.csv", 100, 4, 0.95)
complete_T_MLE(P_small)
U, policy = value_iteration(P_small, 1000)
# output policy
outfile = "policy/small.policy"
open(outfile, "w") do io
    for s=1:P_small.S
        a_s, U_s = lookahead(P_small, U, s)
        @printf(io, "%d\n", a_s)
    end
end

Medium.csv
========================

In [4]:
# mountain car scenario
# this scenario has two characteristics
# 1. the reward function is almost always entirely determined by the action
#    1 -> -225, 2 -> -100, 3 -> -25, 4 -> 0, 5 -> -25, 6 -> -100, 7 -> -225
#    except there are some flags somewhere that carries a reward of 100,000
# 2. the transition is also (almost) deterministic
#    v_{t + 1} = v_t + p * force - lambda * cos(3 * pos)
#    p_{t + 1} = p_t + mu * v_{t + 1} <- this is rounded randomly if v_{t + 1} is not an integer
#    where p, lambda, mu are some parameters
#    except at the boundary where there is inelastic collision

# the force corresponding to action
force = [-15, -10, -5, 0, 5, 10, 15]

# get (pos, vel) from state
function get_p_v(s)
    return (s - 1) % 500, fld(s - 1, 500)
end
# first let's find the position of the flag
df_med = CSV.read("data/medium.csv", DataFrame)
# add p and v to the dataframe for convenience
df_med.p = (df_med.s .- 1) .% 500
df_med.v = (df_med.s .- 1) .÷ 500 .- 50  # the speed is actually normalized
df_med.pp = (df_med.sp .- 1) .% 500
df_med.vp = (df_med.sp .- 1) .÷ 500 .- 50
# find flags

df_flag = df_med[df_med.r.>0, :]
flags = Set()
for row in eachrow(df_flag)
    push!(flags, row.p)
end
# it looks like the flag is in positive 454 -- 460
@printf("flag: %s\n", flags)
# let's now interpolate the parameters of the program

df_med.dv = df_med.vp .- df_med.v
df_med.dp = df_med.pp .- df_med.p

df_med

flag: Set(Any[456, 454, 460, 455, 457, 459, 458])


Row,s,a,r,sp,p,v,pp,vp,dv,dp
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,25203,1,-225,24703,202,0,202,-1,-1,0
2,24703,3,-25,24703,202,-1,202,-1,0,0
3,24703,1,-225,24202,202,-1,201,-2,-1,-1
4,24202,1,-225,24202,201,-2,201,-2,0,0
5,24202,2,-100,23701,201,-2,200,-3,-1,-1
6,23701,3,-25,23700,200,-3,199,-3,0,-1
7,23700,1,-225,23699,199,-3,198,-3,0,-1
8,23699,3,-25,23198,198,-3,197,-4,-1,-1
9,23198,4,0,23697,197,-4,196,-3,1,-1


In [5]:
df_med[df_med.sp .== 26342, :]

Row,s,a,r,sp,p,v,pp,vp,dv,dp
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,27341,5,-25,26342,340,4,341,2,-2,1
2,27342,4,0,26342,341,4,341,2,-2,0


In [6]:
using Plots, Statistics
# first, figure out coefficient between vp and dp. 
df_med_san = df_med[df_med.s .!= 25343, :]
# check that they are correlated
vp = df_med_san.vp
dp = df_med_san.dp
corr = sum(vp .* dp) / (sum(vp .* vp) * sum(dp .* dp))^0.5
@printf("correlation: %f\n", corr)
# nice correlation. We can compute the beta
beta = sum(vp .* dp) / sum(vp .* vp)
@printf("beta: %f\n", beta)
# the beta is around 0.352475. So dp is around 0.352 * vp
df_med_san.dp_predicted = df_med_san.vp * beta
# ok let's now compute the coefficient that determines dv
# normalize a column
df_med_san.F = df_med_san.a .- 4
# to guess the slope of the curve, we focus those data with F = 0
df_med_no_force = df_med_san[df_med_san.F .== 0, :]
# Since the slope is entirely a function of position, we can interpolate it by aggregating.
# there is a wall at 0 and the process ends past 460
slope = ones(500)
for p=20:460
    df_at_p = df_med_no_force[(df_med_no_force.p .> p - 5) .& (df_med_no_force.p .< p + 5), :]
    slope[p] = mean(df_at_p.dv)
end
for p=1:20
    slope[p] = slope[20]
end
plot(collect(1:460), slope)
# the plot looks quite accurate
# judging from the plot, the following should roughly be the slope function
# note that the slop array is not accurate at the endpoint 0 as there is collison
# let's now figure out the effect of force
df_med_san_mid = df_med_san[df_med_san.p .> 100 .& df_med_san.p .< 400, :]
df_med_san_mid[!, :dv_F] = df_med_san_mid.dv - [slope[p] for p in df_med_san_mid.p]
df_med_san_mid
dv_F = df_med_san_mid.dv_F
F = df_med_san_mid.F
corr = sum(dv_F .* F) / (sum(dv_F .* dv_F) * sum(F .* F))^0.5
@printf("correlation: %f\n", corr)
beta = sum(dv_F .* F) / sum(F .* F)
@printf("beta: %f\n", beta)
Pi = 3.1415926
# a empirical good model for dv
function dv_emp(p, F)
    return 0.25 * F + 2.0 * cos(2 * Pi * (p - 50) / 590)
end

df_med_san.dv_emp = [dv_emp(r.p, r.F) for r in eachrow(df_med_san)]

corr = sum(df_med_san.dv_emp .* df_med_san.dv) / (sum(df_med_san.dv_emp .* df_med_san.dv_emp) * sum(df_med_san.dv .* df_med_san.dv))^0.5
@printf("corr of dv prediction: %f\n", corr)
df_med_san



correlation: 0.996235
beta: 0.353095
correlation: 0.425796
beta: 0.229774
corr of dv prediction: 0.750388


Row,s,a,r,sp,p,v,pp,vp,dv,dp,dp_predicted,F,dv_emp
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Float64
1,25203,1,-225,24703,202,0,202,-1,-1,0,-0.353095,-3,-0.845808
2,24703,3,-25,24703,202,-1,202,-1,0,0,-0.353095,-1,-0.345808
3,24703,1,-225,24202,202,-1,201,-2,-1,-1,-0.706191,-3,-0.845808
4,24202,1,-225,24202,201,-2,201,-2,0,0,-0.706191,-3,-0.824529
5,24202,2,-100,23701,201,-2,200,-3,-1,-1,-1.05929,-2,-0.574529
6,23701,3,-25,23700,200,-3,199,-3,0,-1,-1.05929,-1,-0.303241
7,23700,1,-225,23699,199,-3,198,-3,0,-1,-1.05929,-3,-0.781947
8,23699,3,-25,23198,198,-3,197,-4,-1,-1,-1.41238,-1,-0.260649
9,23198,4,0,23697,197,-4,196,-3,1,-1,-1.05929,0,0.0106495


In [9]:
using DataFrames, GLM, StatsBase, LinearAlgebra
# Try doing approximate value iteration
# generate feature columns in dataframe 
# "THIS PROBLEM IS UNDISCOUNTED, AND THE PROCESS ENDS AT THE FLAG"

# reward function
function reward(p, a)
    flag = 0
    if p >= 454
        flag = 100000 
    end
    return flag - 25 * (a - 4)^2
end

# transition function interpolated from data
# the transition is deterministic
function transition(p, v, a)
    F = a - 4
    vp = v + dv_emp(p, F)
    pp = p + 0.353 * vp 
    # wall collision
    if pp < 0
        pp, vp = 1, 0
    end
    return (pp, vp)
end

# generate feature for a given df
# assume df must have columns p, v
# we also normalize these p and v a bit
F_LEN = 21 # parameter, number of features
function make_feature(p, v)
    p_n = p / 500
    v_n = v / 50
    feature =  (1, 
                p_n, v_n, 
                p_n^2, p_n*v_n, v_n^2, 
                p_n^3, p_n^2*v_n, p_n*v_n^2, v_n^3, 
                v_n^4, p_n^4, p_n^3*v_n, p_n*v_n^3, p_n^2 * v_n^2,
                v_n^5, p_n^5, p_n^4*v_n, p_n^3*v_n^2, p_n^2 * v_n^3, p_n * v_n^4)
    @assert size(feature, 1) == F_LEN
    return feature
end

# add the features to the dataframe
function add_feature_df(df)
    for i=1:F_LEN
        col_name = 'f'*string(i)
        df[!, col_name] = [make_feature(r.p, r.v)[i] for r in eachrow(df)]
    end
    return df
end

# generate the greedy action w.r.t the reward function defined by theta
# row is a row vector, where (row.p, row.v) is the current states
# theta is the parameter
function greedy(row, theta)
    @assert size(theta, 1) == F_LEN
    best_a, best_r = 0, -Inf
    p, v = row.p, row.v
    for a=1:7
        pp, vp = transition(p, v, a)
        if reward(p, a) < 0
            r = reward(p, a) + dot(theta, make_feature(pp, vp))
        else
            # if we don't hit flag, we stop earning reward
            r = reward(p, a)
        end
        if r > best_r
            best_a, best_r = a, r
        end
    end
    return best_a, best_r
end

# update the parameters to reflect 
# theta denotes the old set of parameters. The utility function is approximated by theta dot 
# returns the new set of 
# df should contain the feature columns, f1 ... fn
function update(theta, df)
    # predict the reward using the 
    df.U_p = [dot(theta, make_feature(r.p, r.v)) for r in eachrow(df)]
    # find the greedy utility
    df.U = [greedy(r, theta)[2] for r in eachrow(df)]
    # print how far the reward is from the actual
    loss = mean((df.U_p .- df.U) .^ 2)
    @printf("Average Square Loss: %f\n", loss)
    # Fit the new parameters, using simple linear regression
    ols = lm(@formula(U ~ f2 + f3 + f4 + f5 + f6 + f7 + f8 + f9 + f10 + f11 + f12 + f13 + f14 + f15 + f16 + f17 + f18 + f19 + f20 + f21), df) # f1 is not here since life 
    return GLM.coef(ols)
end

# initialize a dataframe consisting of p, v and the feature vectors
df = DataFrame(s=1:50000)
df.p = (df.s .- 1) .% 500
df.v = (df.s .- 1) .÷ 500 .- 50 
add_feature_df(df)
# we start with a degree 4 model
theta = [95210.32621926059, 3167.834252108287, 681.3966194715723, -33975.77329677989, 3823.0941527381738, 2157.0855349683798, 37281.21716333439, -3298.468309221864, -1936.1267282317328, -450.0528392257383, 
    0,0,0,0,0,0,0,0,0,0,0]
for iter=1:1000
    theta = update(theta, df)
    if iter % 50 == 0
        @printf("Iteration: %d\n", iter)
        @printf("%s\n", theta)
        @printf("==============================")

    end
end
@printf("%s\n", theta)
# after about 1000 iterations it converges to the following theta
# [94741.53636256832, 3411.463010466631, 719.241251454842, -35853.044875943466, 4061.901747398426, 2317.842042553612, 39495.13870829302, -3496.915870571967, -2075.8165640408633, -471.7311047696814]
# an even better theta after about 2000 iterations
# [95210.32621926059, 3167.834252108287, 681.3966194715723, -33975.77329677989, 3823.0941527381738, 2157.0855349683798, 37281.21716333439, -3298.468309221864, -1936.1267282317328, -450.0528392257383]
# maybe next time use a more complicated model

# this needs serious improvements, to at least 110

Average Square Loss: 194140.463901
Average Square Loss: 135760.746161
Average Square Loss: 131681.317680
Average Square Loss: 140124.993700
Average Square Loss: 155106.921408
Average Square Loss: 173307.627083
Average Square Loss: 192221.513884
Average Square Loss: 209768.853274
Average Square Loss: 224252.002785
Average Square Loss: 234447.167726
Average Square Loss: 239686.582401
Average Square Loss: 239866.962922
Average Square Loss: 235361.497058
Average Square Loss: 226884.663234
Average Square Loss: 215355.707454
Average Square Loss: 201772.352604
Average Square Loss: 187092.129020
Average Square Loss: 172141.727672
Average Square Loss: 157571.840281
Average Square Loss: 143835.748920
Average Square Loss: 131181.450443
Average Square Loss: 119643.477150
Average Square Loss: 109131.462956
Average Square Loss: 99489.920042
Average Square Loss: 90553.962524
Average Square Loss: 82195.614533
Average Square Loss: 74352.922175
Average Square Loss: 67037.329026
Average Square Loss: 6032

LoadError: InterruptException:

In [39]:
# output policy
outfile = "policy/medium.policy"
open(outfile, "w") do io
    for s=1:50000
        p = (s - 1) % 500
        v = (s - 1) ÷ 500 - 50
        row = DataFrame(p=p, v=v)
        a_s, U_s = greedy(row[1,:], theta)
        @printf(io, "%d\n", a_s)
    end
end

In [35]:
# Maybe simply trying Bellman update here would be nicer?
# transition function
# here we discretize the states, 
# TBD...
function transition_discrete(p, v, a)
    F = a - 4
    vp = v + 0.3 * F + 2 * cos(2 * 3.1415 * (p - 50) / 600)
    pp = p + 0.353 * vp 
    # 
    if pp < 0
        pp, vp = 1, 0
    end
    return (pp, vp)
end

Large.CSV
==============

In [11]:
# let's large.csv 

df_large = CSV.read("data/large.csv", DataFrame)
@printf("state space: %d\n", maximum(df_large.s))
@printf("action space: %d\n", maximum(df_large.a))
# the state space seems to have size 302020
# the action space have size 9
# glacing from the data, it looks like the state space can be divided into three parts
function split(s)
    return s 
end
s = df_large.s
df_large.s1 = s.÷ 10000
df_large.s2 = (s.%10000).÷100
df_large.s3 = s.%100
sp = df_large.sp
df_large.sp1 = sp.÷ 10000
df_large.sp2 = (sp.%10000).÷100
df_large.sp3 = sp.%100
# it looks like the first coord only takes 5 values
# and the second, third coord only takes 10 values
# so the state space has size 500???
S1Range = [15,23,27,29,30]
S2Range = [1,2,3,4,10,11,12,13,14,20]
S3Range = [1,2,3,4,10,11,12,13,14,20]

S1Code = Dict()
for i=1:size(S1Range,1)
    S1Code[S1Range[i]] = i - 1
end
S2Code = Dict()
for i=1:size(S2Range,1)
    S2Code[S2Range[i]] = i - 1
end
S3Code = Dict()
for i=1:size(S3Range,1)
    S3Code[S3Range[i]] = i - 1
end

# ok i'm just going to map the state space to 1-500 and use Bellman update
function encode(s)
    s1 = s ÷ 10000
    s2 = (s%10000)÷100
    s3 = s%100
    return 100 * S1Code[s1] + 10 * S2Code[s2] + S3Code[s3] + 1
end
df_large_encoded = DataFrame("s" => [encode(s) for s in df_large.s], "a" => df_large.a,
                             "r" => df_large.r, "sp" => [encode(sp) for sp in df_large.sp])

# actions?
# alright it seems 1,2,3,4 are standard grid world actions
# where as 5,6,7,8,9 "jumps between layers" seemingly randomly with very low probability of succeeding
# (See below for the description of the teleporters)

df_large_encoded[(df_large_encoded.r .> 0), :]
# ok I guess I can just use value iteration
# this is a dedicated version that is hand-tuned to this particular map
p_dir = [(1, 0), (-1, 0), (0, -1), (0, 1)]
function load_from_df_large(df, S, A, γ)
    # initialize the sample and reward array
    samples = Dict()
    rewards = Dict()
    for s=1:S
        for a=1:A
            samples_s_a = Dict()
            df_s_a = df[(df.s.==s) .& (df.a.==a), :]
            if size(df_s_a, 1) > 0
                df_s_a_count = combine(groupby(df_s_a, :sp), nrow)
                for i=1:size(df_s_a_count,1)
                    samples_s_a[df_s_a_count[i, :sp]] = df_s_a_count[i,:nrow]
                end
                samples[(s, a)] = samples_s_a
            end
        end
    end
    # the reward function here is a function of (sp, a) instead of (s, a)
    for sp=1:S
        for a=1:A
            samples_sp_a = Dict()
            df_sp_a = df[(df.sp.==sp) .& (df.a.==a), :]
            if size(df_sp_a, 1) > 0
                rewards[(sp, a)] = df_sp_a[1, :r]
                # check that this is indeed the correct assumption
                for row in eachrow(df_sp_a)
                    @assert row.r == df_sp_a[1, :r]
                end
            end
        end
    end
    # there might be a few missing entries
    for sp=1:S
        for a=1:A
            if !((sp, a) in keys(rewards))
                # I know that a >= 5 always give reward 0
                if a >= 5
                    rewards[(sp, a)] = 0
                else
                    # otherwise, hope that there is data for some other a
                    ap = 5 - a
                    rewards[(sp, a)] = rewards[(sp, ap)]
                end
            end
        end
    end
    return MDP(γ, S, A, Dict(), rewards, samples)
end

# completing the Transition matrix
# fine_tuned to particular behavior of this instance
function complete_T_large(P::MDP)
    T = Dict()
    for s=1:P.S
        for a=1:P.A
            # extract coordinates
            l = (s - 1)÷100 + 1 # l ranges from 1-5
            x = (s - 1)%10 + 1 # x ranges from 1-10
            y = ((s - 1)÷10)%10 + 1 # y also ranges from 1-10
            # non-teleporter actions
            # assume they behave identically on each layer
            # we now make the same assumption that they behave identically at each square
            if a <= 4
                T_s_a = Dict()
                p_dir = [(1, 0), (-1, 0), (0, -1), (0, 1)] # the "principal direction" of each action
                p_err = [0.20, 0.30, 0.20, 0.25, 0.10] # the probability that an action makes an error
                # this looks like a good heuristic, but not optimal still
                for dir in p_dir
                    # calculate the destination if we step in dir
                    xp, yp = x + dir[1], y + dir[2]
                    # boundary collision
                    if (xp < 1) | (xp > 10) | (yp < 1) | (yp > 10)
                        xp, yp = x, y
                    end
                    sp = (l - 1) * 100 + (yp - 1) * 10 + (xp - 1) + 1
                    if dir == p_dir[a]
                        prob = 1 - p_err[l]
                    else
                        prob = p_err[l] / 3
                    end
                    if !(sp in keys(T_s_a))
                        T_s_a[sp] = 0
                    end
                    T_s_a[sp] += prob
                end
                T[(s,a)] = T_s_a
            else 
                # not a teleporter spot
                if !((s % 100) in [0,1,10,91])
                    T[(s,a)] = Dict(s => 1.0)
                else
                    # I dont really know what to do in this case
                    # I'll go with simple empirical evidence for now
                    # ok, based on further study
                    # it seems that there is about 20% probability that one is teleported to some
                    # random spot not in action_layer
                    # and 80% probability that one is teleported to some random spot in action_layer
                    action_layer = a - 4
                    if (action_layer == l)
                        T[(s,a)] = Dict(s => 1.0)
                    else
                        T[(s,a)] = Dict()
                        for lp=1:5
                            for sp=(100*lp-99):(100*lp)
                                # empirical probability of failed teleported
                                p_fail = 136 / 450
                                # there seem to be a broken teleporter at s = 401
                                # but including this seems to be a bad idea
                                #if (s == 401)
                                #    p_fail = 0.75
                                #end
                                if (lp == action_layer)
                                    T[(s, a)][sp] = (1 - p_fail) / 100
                                # it looks like wrong teleportation 
                                # always sends us to a different floor
                                elseif (lp != l)
                                    T[(s, a)][sp] = p_fail / 300
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    P.T = T
end

# one step lookahead, based on a certain utility function
# given MDP P and utility U, output best policy and utility from state s
# adapted for utility function depending on (sp, a) instead of (s, a)
function lookahead_large(P::MDP, U, s)
    Q_s = Dict()
    for a=1:P.A
        Q_s[a] = sum([prob*(P.R[(sp,a)] + P.γ * U[sp]) for (sp, prob) in P.T[(s,a)]])
    end
    # update
    a_s, U_s = -1, -Inf
    for a=1:P.A
        if Q_s[a] > U_s
            a_s = a
            U_s = Q_s[a]
        end
    end
    return a_s, U_s 
end

# solving large via value iteration
function value_iteration_large(P::MDP, iter::Int64)
    # initialize utility function
    # utility function
    U = Dict()
    for s=1:P.S
        U[s] = 0
    end
    # bellman update
    # looka
    
    for it=1:iter
        for s=1:P.S
            # compute action-value funciton
            a_s, U_s = lookahead_large(P,U,s)
            U[s] = U_s
        end
    end

    # output policy based on utility
    policy = Dict()
    for s=1:P.S
        a_s, U_s = lookahead_large(P,U,s)
        policy[s] = a_s
    end

    return U, policy
end


# Now I need to change everything thanks to this wonderful reward function...

# i think to progress further, i need to understand how the actions 1,2,3,4 behave better.
# maybe they act uniformly on all squares, who knows?
train = true
P_large = load_from_df_large(df_large_encoded, 500, 9, 0.95)
complete_T_large(P_large)
#@printf("%s\n", P_large.T[301, 4])
#@printf("%s\n", P_large.T[205, 4])
#@printf("%s\n", P_large.T[301, 5])
if train
    # there is a single missing entry in the reward table, that I just guess 23333
    U, policy = value_iteration_large(P_large, 2000)
    @printf("%s\n", [U[s] for s=490:500])
end

state space: 302020
action space: 9
[531.8176298023416, 565.2033643264537, 531.8176969328878, 500.4401226673907, 470.9757125136013, 444.1539282914299, 444.1511237575558, 470.9738837860758, 500.4395002811545, 531.817629722115, 565.2033643264537]


In [44]:
outfile = "policy/large.policy"
open(outfile, "w") do io
    # first compute the policy from all the reachable states
    reachable = Dict()
    for encoded_s=1:500
        # compute the optimal action
        a_s = policy[encoded_s]
        # unencode the state
        s1 = S1Range[(encoded_s - 1) ÷ 100 + 1]
        s2 = S2Range[((encoded_s - 1) ÷ 10) % 10 + 1]
        s3 = S3Range[(encoded_s - 1) % 10 + 1]
        reachable[s1*10000+s2*100+s3] = a_s
    end
    for s=1:312020
        if s in keys(reachable)
            @printf(io, "%d\n", reachable[s])
        else
            @printf(io, "%d\n", 1)
        end
    end
end

In [56]:
# look at teleporter behavior

df_teleport = df_large_encoded[(df_large_encoded.a .> 4) .&& (df_large_encoded.s .!= df_large_encoded.sp), :]
df_teleport.l = (df_teleport.s .- 1) .÷ 100 .+ 1
df_teleport.lp = (df_teleport.sp .- 1) .÷ 100 .+ 1
#@printf("%s\n", teleport_success_locations)
# it looks like there are exact 20 spots where the teleporter can succeed
# [301, 401, 391, 200, 110, 291, 491, 500, 1, 300, 100, 310, 101, 10, 410, 210, 400, 91, 201, 191]
# a.k.a the teleporters look like they are on the corners of the map
# at each teleporter, there is a single action that fixes the action, while the other actions just teleport the bot to a random position?
# teleporter never earns you anything
#df_large_encoded[(df_large_encoded.a .== 9) .&& (df_large_encoded.s .== 333), :]

# it looks like layer 1 has a bunch of treasures :-)
# maybe you do want to teleport when you are in other layer

# ok so the strategy might be: if you are not in layer 1 rush to a teleporter ASAP, otherwise stick in layer 1
# Also I have an inchling that 1,2,3,4 behave identically on each layer
# Ok I think 5,6,7,8,9 has a higher propensity to teleport you to layer 1,2,3,4,5 respectively
# sum up the teleportation frequency of teleporting to the correct floor
# maybe the teleporter success rate also differs by floor?
num_tele = 0
num_fail = 0
for a in [5,6,7,8,9]
    for tele in [0, 1, 10, 91]
        df_tele = df_l_e[(df_l_e.a .== a) .&& (df_l_e.s .% 100 .== tele) .&& (df_l_e.sp .!= df_l_e.s), :]
        num_tele += size(df_tele, 1)
        num_fail += size(df_tele[(df_tele.sp .- 1) .÷ 100 .!= (a - 5), :], 1)
    end
end
@printf("%d, %d\n", num_tele, num_fail)

# df_teleport[(df_teleport.sp .- 1) .÷ 100 .!= (df_teleport.a .- 5), :]
# Ok I'm calling it off here, since i don't think teleporter behavior would affect the policy at all
# as is, the only sensible thing to do at teleporters is to just press 5 and hope for the best

# maybe something else to consider is that the teleporters might be different
for tele in [0, 1, 10, 91]
    df_tele = df_teleport[(df_teleport.s .% 100 .== tele), :]
    num_tele = size(df_tele, 1)
    num_fail = size(df_tele[df_tele.lp .!= (df_tele.a .- 4), :], 1)
    @printf("teleporter at %d, total %d, fail %d\n", tele, num_tele, num_fail)
end

for l in [1,2,3,4,5]
    df_tele = df_teleport[(df_teleport.l .== l), :]
    num_tele = size(df_tele, 1)
    num_fail = size(df_tele[df_tele.lp .!= (df_tele.a .- 4), :], 1)
    @printf("teleporter at layer %d, total %d, fail %d\n", l, num_tele, num_fail)
end

for tele in [0, 1, 10, 91]
    for l in [1,2,3,4,5]
        df_tele = df_teleport[(df_teleport.s .% 100 .== tele) .&& (df_teleport.l .== l), :]
        num_tele = size(df_tele, 1)
        num_fail = size(df_tele[df_tele.lp .!= (df_tele.a .- 4), :], 1)
        expected_fail = num_tele * 136/450
        @printf("teleporter at (%d, %d), total %d, fail %d, efail %d\n", l, tele, num_tele, num_fail, expected_fail)
    end
end

# hmm it looks like there is a very bad teleporter at (5, 1)

df_teleport[df_teleport.s .== 401, :]

450, 136
teleporter at 0, total 110, fail 27
teleporter at 1, total 104, fail 37
teleporter at 10, total 133, fail 43
teleporter at 91, total 103, fail 29
teleporter at layer 1, total 69, fail 16
teleporter at layer 2, total 105, fail 34
teleporter at layer 3, total 98, fail 29
teleporter at layer 4, total 92, fail 25
teleporter at layer 5, total 86, fail 32
teleporter at (1, 0), total 19, fail 4, efail 6
teleporter at (2, 0), total 27, fail 6, efail 8
teleporter at (3, 0), total 23, fail 4, efail 7
teleporter at (4, 0), total 22, fail 5, efail 7
teleporter at (5, 0), total 19, fail 8, efail 6
teleporter at (1, 1), total 18, fail 3, efail 5
teleporter at (2, 1), total 24, fail 9, efail 7
teleporter at (3, 1), total 23, fail 7, efail 7
teleporter at (4, 1), total 23, fail 6, efail 7
teleporter at (5, 1), total 16, fail 12, efail 5
teleporter at (1, 10), total 17, fail 6, efail 5
teleporter at (2, 10), total 38, fail 14, efail 11
teleporter at (3, 10), total 28, fail 9, efail 8
teleporte

Row,s,a,r,sp,l,lp
,Int64,Int64,Int64,Int64,Int64,Int64
1,401,7,0,335,5,4
2,401,7,0,137,5,2
3,401,6,0,115,5,2
4,401,5,0,61,5,1
5,401,8,0,77,5,1
6,401,8,0,188,5,2
7,401,8,0,171,5,2
8,401,7,0,25,5,1
9,401,7,0,313,5,4


In [11]:
# more detailed study of the behavior of action 1,2,3,4

# hypothesis I: they act identically on each layers
df_l_e = df_large_encoded
df_l_e.l = (df_l_e.sp .- 1) .÷ 100
df_l_e.df = df_l_e.sp - df_l_e.s
principal = [1, -1, -10, 10]
for a in [1,2,3,4]
    total_move = 0
    total_principal = 0
    for l in [1,2,3,4,5]
        df_a_l = df_l_e[(df_l_e.a .== a) .&& (df_l_e.sp .!= df_l_e.s) .&& ((df_l_e.sp .- 1) .÷ 100 .== (l - 1)), :]
        df_a_l.ds = df_a_l.sp .- df_a_l.s
        ttl = size(df_a_l, 1)
        mvs = Dict()
        # compute the number of steps in each direction
        df_grp = groupby(df_a_l, :ds)
        for sub_df in df_grp
            mvs[sub_df[1, :ds]] = size(sub_df, 1) / ttl
        end
        @printf("a %d, l %d: %s\n", a, l, mvs)
        total_move += ttl
        total_principal += mvs[principal[a]] * ttl
    end
    @printf("total principal probability of %d: %f\n", a, total_principal / total_move)
end
# ok here is a hypothesized model
# 1, 2, 3, 4 move the bot in principal direction  1, -1, -10, 10 respectively
# they succeed with probability roughly 0.75, and move in some other direction otherwise
# (On layer 5, they succeed with slightly higher probability of 0.9)

# I guess I need to analyze the spacial behavior of moves more carefully

a 1, l 1: Dict{Any, Any}(-1 => 0.0730593607305936, 10 => 0.06506849315068493, -10 => 0.07134703196347032, 1 => 0.7905251141552512)
a 1, l 2: Dict{Any, Any}(-1 => 0.09237758707723372, 10 => 0.09540636042402827, -10 => 0.09742554265522463, 1 => 0.7147905098435133)
a 1, l 3: Dict{Any, Any}(-1 => 0.07221976074435091, 10 => 0.07133362871067789, -10 => 0.06202924235711121, 1 => 0.79441736818786)
a 1, l 4: Dict{Any, Any}(-1 => 0.07799564270152505, 10 => 0.08366013071895424, -10 => 0.08496732026143791, 1 => 0.7533769063180827)
a 1, l 5: Dict{Any, Any}(-1 => 0.03662420382165605, 10 => 0.029193205944798302, -10 => 0.029193205944798302, 1 => 0.9049893842887473)
total principal probability of 1: 0.789458
a 2, l 1: Dict{Any, Any}(-1 => 0.8055872291904219, 10 => 0.062143671607753706, -10 => 0.06727480045610035, 1 => 0.06499429874572406)
a 2, l 2: Dict{Any, Any}(-1 => 0.683742812336644, 10 => 0.10402509147935181, -10 => 0.11238891792995295, 1 => 0.09984317825405123)
a 2, l 3: Dict{Any, Any}(-1 => 0.8

LoadError: syntax: incomplete: premature end of input

In [42]:
#118, 128
df_ww = df_l_e[(df_l_e.a .<= 4) .&& (df_l_e.sp .== 10), :]


Row,s,a,r,sp,l,df
,Int64,Int64,Int64,Int64,Int64,Int64
1,9,2,0,10,0,1
2,20,3,0,10,0,-10
3,20,3,0,10,0,-10
4,9,1,0,10,0,1
5,20,3,0,10,0,-10
6,20,3,0,10,0,-10
7,9,1,0,10,0,1
8,9,1,0,10,0,1
9,20,2,0,10,0,-10
